# 動画からデータセットを作成しよう！

### .MOV converto to .MP4

In [19]:
# option
# ffmpeg -i current_video_name.mov new_file_name.mp4

import cv2
import os
from tqdm import tqdm 
import shutil
root_dir = os.curdir
print(root_dir)
video_path = './IMG_4515.mov' 
output_folder = 'dataset' 
set_folder = 'result_preview_dataset'
# setting time
time_setting = 2.0

def resize_image(image):
    #(h,w)= image.shape[:2]
    resized_image = cv2.resize(image,None,fx=0.05,fy=0.05)
    return resized_image

if not os.path.exists(output_folder):
    os.makedirs(output_folder)
else:
    shutil.rmtree(output_folder)
    os.makedirs(output_folder)
    
video_frame = cv2.VideoCapture(video_path)

# 동영상의 FPS 값 가져오기
fps = video_frame.get(cv2.CAP_PROP_FPS)
if fps == 0:
    fps = 30  # FPS 정보를 가져올 수 없을 때 기본값 설정

# 프레임 간격 계산
frame_interval = int(fps * time_setting)

frame_count = 0  # current frame number
image_count = 0  # saved image
total_frames = int(video_frame.get(cv2.CAP_PROP_FRAME_COUNT))
with tqdm(total=total_frames, desc="Processing") as pbar:
    while video_frame.isOpened():
        ret, frame = video_frame.read()
        if not ret:
            break

        if frame_count % frame_interval == 0:
            image_name = f"{image_count}.jpeg"
            save_path = os.path.join(output_folder, image_name)
            
            # 이미지 저장
            cv2.imwrite(os.path.join(output_folder, image_name), frame)
            
            #resized_frame = cv2.resize(frame, (320, 180))
            #resized_frame = resize_image(frame)
            #cv2.imwrite(os.path.join(preview_dataset_folder, image_name), resized_frame)
            
            image_count += 1

        frame_count += 1
        pbar.update(1)  # 진행 상황 업데이트
# 자원 해제
video_frame.release()
cv2.destroyAllWindows()

In [1]:
from PIL import Image
import piexif
import cv2
import os
from tqdm import tqdm
import shutil

video_path = './park1/VID_20250517_121028.mp4'
output_folder = 'dataset'
video_corp = 'POCO'
video_equi = 'F7ultra'
video_focal = (53, 10)  # 5.3mm
time_setting = 0.3

if not os.path.exists(output_folder):
    os.makedirs(output_folder)
else:
    shutil.rmtree(output_folder)
    os.makedirs(output_folder)

video_frame = cv2.VideoCapture(video_path)

# 동영상의 FPS 값 가져오기
fps = video_frame.get(cv2.CAP_PROP_FPS)
if fps == 0:
    fps = 30  # FPS 정보를 가져올 수 없을 때 기본값 설정

# 프레임 간격 계산
frame_interval = int(fps * time_setting)

frame_count = 0  # current frame number
image_count = 0  # saved image
total_frames = int(video_frame.get(cv2.CAP_PROP_FRAME_COUNT))

# EXIF 데이터 설정
exif_dict = {
    "0th": {
        piexif.ImageIFD.Make: video_corp,
        piexif.ImageIFD.Model: video_equi
    },
    "Exif": {
        piexif.ExifIFD.FocalLength: video_focal
    }
}
exif_bytes = piexif.dump(exif_dict)

with tqdm(total=total_frames, desc="Processing") as pbar:
    while video_frame.isOpened():
        ret, frame = video_frame.read()
        if not ret:
            break

        if frame_count % frame_interval == 0:
            image_name = f"{image_count}.jpeg"
            save_path = os.path.join(output_folder, image_name)

            # OpenCV는 EXIF를 유지하지 못하므로 PIL로 변환하여 저장
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # BGR -> RGB 변환
            pil_image = Image.fromarray(image)

            # EXIF 정보를 추가하여 저장
            pil_image.save(save_path, "jpeg", exif=exif_bytes)
            
            image_count += 1

        frame_count += 1
        pbar.update(1)

# 자원 해제
video_frame.release()
cv2.destroyAllWindows()

Processing: 100%|██████████| 10212/10212 [01:17<00:00, 131.70it/s]
